In [1]:
!pip install -q -r requirements.txt

In [2]:
from datasets import load_dataset

In [3]:
# Load in all datasets
chronic_ds = load_dataset("Bhawna/ChroniclingAmericaQA", split="train")
timeqa_ds = load_dataset("hugosousa/TimeQA", split="train")

============ LOADING DATASETS ============
============ LOADING CHRONICQA ============
============ LOADING TIMEQA ============
